## 1. Prepare Dataset

In [1]:
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader

In [2]:
batch_size = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081, ))
])

In [3]:
X_train = datasets.MNIST(root='../dataset/mnist/', train=True, download=False, transform=transform)
X_test = datasets.MNIST(root='../dataset/mnist/', train=False, download=False, transform=transform)
train_loader = DataLoader(X_train, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(X_test, shuffle=True, batch_size=batch_size)

## 2. Design Model

In [11]:
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320, 10) # 全连接
    def forward(self, x):
        batch_size = x.size(0)
        x = self.pooling(F.relu(self.conv1(x)))
        x = self.pooling(F.relu(self.conv2(x)))
        x = x.view(batch_size, -1)
        x = self.fc(x)
        return x

In [19]:
model = Net()
# 加入 GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
# 加入 GPU

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pooling): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=320, out_features=10, bias=True)
)

## 3. Construct loss and optimizer

In [20]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [21]:
def train(epoch):
    running_loss = 0.0
    for batch_index, data in enumerate(train_loader, 0):
        inputs, target = data
        # 加入 GPU
        inputs, target = inputs.to(device), target.to(device)
        # 加入 GPU
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    if(batch_index % 300 == 299):
        print("[%d, %5d] loss: %.3f" % (epoch + 1, batch_index + 1, running_loss / 300))
        running_loss = 0.0

In [22]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, target = data
            # 加入 GPU
            inputs, target = inputs.to(device), target.to(device)
            # 加入 GPU
            outputs = model(inputs)
            _, predict = torch.max(outputs.data, dim=1)
            total += target.size(0)
            correct += (predict == target).sum().item()
    print("Accuracy on test set: %d %% [%d/%d]" % (100 * correct / total, correct, total))

In [24]:
%%time
if __name__ == "__main__":
    for epoch in range(10):
        train(epoch)
        test()

Accuracy on test set: 98 % [9880/10000]
Accuracy on test set: 98 % [9890/10000]
Accuracy on test set: 98 % [9891/10000]
Accuracy on test set: 98 % [9887/10000]
Accuracy on test set: 98 % [9893/10000]
Accuracy on test set: 98 % [9893/10000]
Accuracy on test set: 98 % [9894/10000]
Accuracy on test set: 98 % [9897/10000]
Accuracy on test set: 98 % [9892/10000]
Accuracy on test set: 98 % [9895/10000]
Wall time: 1min 48s
